In [1]:
!pip install pandas


In [2]:
import pandas as pd
import numpy as np

### Data Cleaning

In [3]:
df = pd.read_table('data/title.basics.tsv.gz')

<ipython-input-3-6569074e42e9>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/title.basics.tsv.gz')


In [4]:
ratings = pd.read_table('data/title.ratings.tsv.gz')

In [5]:
crew = pd.read_table('data/title.crew.tsv.gz')

In [6]:
name = pd.read_table('data/name.basics.tsv.gz')
name = name[['nconst','primaryName']]

In [7]:
movies = df.query('titleType == "movie"')

In [8]:
crew = crew.drop('writers',axis=1)
movies = movies.merge(ratings,how='left',on='tconst')
movies = movies.merge(crew,how='left',on='tconst')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705510 entries, 0 to 705509
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          705510 non-null  object 
 1   titleType       705510 non-null  object 
 2   primaryTitle    705508 non-null  object 
 3   originalTitle   705508 non-null  object 
 4   isAdult         705510 non-null  object 
 5   startYear       705510 non-null  object 
 6   endYear         705510 non-null  object 
 7   runtimeMinutes  705510 non-null  object 
 8   genres          705510 non-null  object 
 9   averageRating   324809 non-null  float64
 10  numVotes        324809 non-null  float64
 11  directors       705510 non-null  object 
dtypes: float64(2), object(10)
memory usage: 64.6+ MB


In [9]:
movies['directors'] = movies['directors'].str.split(',')

In [10]:
movies = movies.explode('directors')

In [11]:
movies = movies.merge(name,how='left',left_on='directors', right_on='nconst')

In [12]:
movies = movies.drop(['endYear','directors','nconst','titleType'],axis=1)

In [13]:
movies = movies.rename({'primaryName':'directors'},axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784064 entries, 0 to 784063
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          784064 non-null  object 
 1   primaryTitle    784062 non-null  object 
 2   originalTitle   784062 non-null  object 
 3   isAdult         784064 non-null  object 
 4   startYear       784064 non-null  object 
 5   runtimeMinutes  784064 non-null  object 
 6   genres          784064 non-null  object 
 7   averageRating   370044 non-null  float64
 8   numVotes        370044 non-null  float64
 9   directors       709032 non-null  object 
dtypes: float64(2), object(8)
memory usage: 59.8+ MB


In [14]:
columns = [column for column in movies.columns if column != 'directors']

movies = movies.groupby(columns).agg({'directors': lambda nomes: ','.join([nome for nome in nomes if pd.notna(nome)]) if len(nomes) > 0 else np.nan}).reset_index()

In [ ]:
columns = ['startYear','runtimeMinutes','genres']
movies[columns] = movies[columns].map(lambda text: text if text != '\\N' else None)

In [ ]:
movies.to_csv('data/movies_data.csv')